In [325]:
from sklearn import datasets,ensemble,metrics
import pdb
import torch
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
from matplotlib.colors import ListedColormap

In [ ]:
def get_MNIST_train():
    
    mnist_train_data = np.zeros([60000,784]) # fiecare linie este o img 28x28
    mnist_train_labels = np.zeros(60000)
    
    # ~ url = 'https://github.com/golbin/TensorFlow-MNIST/tree/master/mnist/data/train-images-idx3-ubyte.gz'
    # ~ r = requests.get(url, allow_redirects=True)
    # ~ open('train-images-idx3-ubyte.gz', 'wb').write(r.content)
    
    # ~ url = 'https://github.com/golbin/TensorFlow-MNIST/tree/master/mnist/datatrain-images-idx3-ubyte.gz'
    # ~ r = requests.get(url, allow_redirects=True)
    # ~ open('train-labels.idx1-ubyte.gz', 'wb').write(r.content)
    
    f = open('train-images.idx3-ubyte','r',encoding = 'latin-1')
    
    g = open('train-labels.idx1-ubyte','r',encoding = 'latin-1')
    
    byte = f.read(16) #4 bytes magic number, 4 bytes nr imag, 4 bytes nr linii, 4 bytes nr coloane
    byte_label = g.read(8) #4 bytes magic number, 4 bytes nr labels
    
    mnist_train_data = np.fromfile(f,dtype=np.uint8).reshape(60000,28,28)
    mnist_train_labels = np.fromfile(g,dtype=np.uint8)
        
    # Conversii pentru a se potrivi cu procesul de antrenare    
    mnist_train_data = mnist_train_data.astype(np.float32)
    mnist_train_labels = mnist_train_labels.astype(np.int64)
        
    return mnist_train_data, mnist_train_labels

def get_MNIST_test():
    
    mnist_test_data = np.zeros([10000,784])
    mnist_test_labels = np.zeros(10000)
    
    f = open('t10k-images.idx3-ubyte','r',encoding = 'latin-1')
    g = open('t10k-labels.idx1-ubyte','r',encoding = 'latin-1')
    
    byte = f.read(16) #4 bytes magic number, 4 bytes nr imag, 4 bytes nr linii, 4 bytes nr coloane
    byte_label = g.read(8) #4 bytes magic number, 4 bytes nr labels
    
    mnist_test_data = np.fromfile(f,dtype=np.uint8).reshape(10000,784)
    mnist_test_labels = np.fromfile(g,dtype=np.uint8)
    
    # Conversii pentru a se potrivi cu procesul de testare    
    mnist_test_data = mnist_test_data.astype(np.float32)
    mnist_test_labels = mnist_test_labels.astype(np.int64)        
    
    return mnist_test_data, mnist_test_labels

In [327]:
from torch.utils.data import Dataset, DataLoader
class DatasetMNIST(Dataset):
    def __init__(self):

        mnist_train_data,mnist_train_labels = get_MNIST_train()

        self.data = mnist_train_data[:200]
        self.labels = mnist_train_labels[:200]

    def __getitem__(self, index):
        data_point = torch.from_numpy(self.data[index, ...])
        label_point = torch.from_numpy(self.labels[index, ...])

        return {'data': data_point, 'label': label_point}
    
    def __len__(self):
        return self.data.shape[0]


In [328]:
import torch.nn as nn
import torch


class ReteaMLP(nn.Module):
    def __init__(self):
        super().__init__()
 
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(28*28, 512)
        # self.drop_x = nn.Dropout(p=0.5)
        self.bn_x = nn.BatchNorm1d(num_features = 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 10)
 
    def forward(self, x):
        x = self.relu(self.fc1(x))
        # x = self.drop_x(x)
        x = self.bn_x(x)
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
 
        return x
 

In [329]:
import numpy as np

mnistdataset = DatasetMNIST()
 
mnist_loader = DataLoader(mnistdataset, batch_size=128, shuffle=True, num_workers=0)
 
simple_CNN = ReteaMLP()
 
loss_function = nn.CrossEntropyLoss(reduction='sum')
 
optim = torch.optim.SGD(simple_CNN.parameters(), lr=1e-4)

In [ ]:
# Bucla de invatare
     
for ep in range(40):
 
    predictii = []
    etichete = []
 
    for batch in mnist_loader:
        data, label = batch['data'], batch['label']
 
        current_predict = simple_CNN.forward(data)
 
        loss = loss_function(current_predict, label)
 
        current_predict = np.argmax(current_predict.detach().numpy(), axis=1)
 
        predictii = np.concatenate((predictii,current_predict))
        etichete = np.concatenate((etichete,label))
 
        optim.zero_grad()
 
        loss.backward()
 
        optim.step()

    scheduler = torch.optim.lr_scheduler.StepLR(optimizer = optim, step_size = 20, gamma=0.1)
 
    acc = np.sum(predictii==etichete)/len(predictii)
    print( 'Acuratetea la epoca {} este {}%'.format(ep+1,acc*100) )


Acuratetea la epoca 1 este 7.000000000000001%
Acuratetea la epoca 2 este 18.5%
Acuratetea la epoca 3 este 33.5%
Acuratetea la epoca 4 este 46.0%
Acuratetea la epoca 5 este 54.50000000000001%
Acuratetea la epoca 6 este 65.0%
Acuratetea la epoca 7 este 74.0%
Acuratetea la epoca 8 este 77.5%
Acuratetea la epoca 9 este 81.5%
Acuratetea la epoca 10 este 83.5%
Acuratetea la epoca 11 este 86.0%
Acuratetea la epoca 12 este 87.0%
Acuratetea la epoca 13 este 88.0%
Acuratetea la epoca 14 este 88.5%
Acuratetea la epoca 15 este 89.0%
Acuratetea la epoca 16 este 90.5%
Acuratetea la epoca 17 este 91.0%
Acuratetea la epoca 18 este 91.0%
Acuratetea la epoca 19 este 91.0%
Acuratetea la epoca 20 este 92.5%
Acuratetea la epoca 21 este 92.0%
Acuratetea la epoca 22 este 91.5%
Acuratetea la epoca 23 este 91.5%
Acuratetea la epoca 24 este 93.0%
Acuratetea la epoca 25 este 93.0%
Acuratetea la epoca 26 este 94.0%
Acuratetea la epoca 27 este 93.5%
Acuratetea la epoca 28 este 92.5%
Acuratetea la epoca 29 este 94.

In [340]:
# Ex 2
from torchvision import transforms


class DatasetMNIST(Dataset):
    def __init__(self):

        mnist_train_data,mnist_train_labels = get_MNIST_train()

        self.data = mnist_train_data[:200]
        self.labels = mnist_train_labels[:200]

        self.transf = transforms.Compose([
            transforms.ToPILImage(), transforms.Resize([224,224]),
            transforms.ToTensor(), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    def __getitem__(self, index):
        data_point = torch.from_numpy(self.data[index, ...])
        label_point = torch.from_numpy(self.labels[index, ...])

        data_point = np.tile(data_point[..., None], (1,1,3))

        data_point = self.transf(data_point)

        label_point = torch.tensor(label_point,dtype=torch.long)

        return {'data': data_point, 'label': label_point}
    
    def __len__(self):
        return self.data.shape[0]

In [341]:
from torchvision import models

cnn = models.alexnet(pretrained=True)
print(cnn)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [342]:
for param in cnn.features.parameters():
    param.requires_grad = False # nu se mai actualizeaza ponderile

cnn.classifier[6] = nn.Linear(4096,10)

In [344]:
mnistdataset = DatasetMNIST()

mnist_loader = DataLoader(mnistdataset, batch_size=128, shuffle=True, num_workers=0)
 
simple_CNN = cnn
 
loss_function = nn.CrossEntropyLoss(reduction='sum')
 
optim = torch.optim.SGD(simple_CNN.parameters(), lr=1e-4)

# Bucla de invatare
     
for ep in range(40):
 
    predictii = []
    etichete = []
 
    for batch in mnist_loader:
        data, label = batch['data'], batch['label']
 
        current_predict = simple_CNN.forward(data)
 
        loss = loss_function(current_predict, label)
 
        current_predict = np.argmax(current_predict.detach().numpy(), axis=1)
 
        predictii = np.concatenate((predictii,current_predict))
        etichete = np.concatenate((etichete,label))
 
        optim.zero_grad()
 
        loss.backward()
 
        optim.step()

    scheduler = torch.optim.lr_scheduler.StepLR(optimizer = optim, step_size = 20, gamma=0.1)
 
    acc = np.sum(predictii==etichete)/len(predictii)
    print( 'Acuratetea la epoca {} este {}%'.format(ep+1,acc*100) )

C:\Users\maria\AppData\Local\Temp\ipykernel_17568\2467148101.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  label_point = torch.tensor(label_point,dtype=torch.long)


Acuratetea la epoca 1 este 14.499999999999998%
Acuratetea la epoca 2 este 13.0%
Acuratetea la epoca 3 este 19.0%
Acuratetea la epoca 4 este 28.000000000000004%
Acuratetea la epoca 5 este 35.5%
Acuratetea la epoca 6 este 30.0%
Acuratetea la epoca 7 este 31.5%
Acuratetea la epoca 8 este 33.0%
Acuratetea la epoca 9 este 29.5%
Acuratetea la epoca 10 este 31.0%
Acuratetea la epoca 11 este 32.5%
Acuratetea la epoca 12 este 34.5%
Acuratetea la epoca 13 este 32.5%
Acuratetea la epoca 14 este 36.5%
Acuratetea la epoca 15 este 36.0%
Acuratetea la epoca 16 este 35.0%
Acuratetea la epoca 17 este 40.0%
Acuratetea la epoca 18 este 36.5%
Acuratetea la epoca 19 este 43.0%
Acuratetea la epoca 20 este 44.5%
Acuratetea la epoca 21 este 41.0%
Acuratetea la epoca 22 este 45.0%
Acuratetea la epoca 23 este 44.5%
Acuratetea la epoca 24 este 46.0%
Acuratetea la epoca 25 este 45.0%
Acuratetea la epoca 26 este 42.0%
Acuratetea la epoca 27 este 46.0%
Acuratetea la epoca 28 este 47.0%
Acuratetea la epoca 29 este 4